In [10]:
import numpy as np
import open3d as o3d
import os 

In [11]:
def filter_shoes_both(pcd,
                      x_range=(-0.25, 0.25),
                      y_range=(-2.0, 3.0),
                      z_range=(0.15, 0.7),
                      plane_threshold=0.01,
                      ransac_n=3,
                      num_iterations=1000,
                      cluster_eps=0.02,
                      cluster_min_points=100,
                      num_clusters=2):
    # 1) ROI 크롭
    bbox = o3d.geometry.AxisAlignedBoundingBox(
        min_bound=[x_range[0], y_range[0], z_range[0]],
        max_bound=[x_range[1], y_range[1], z_range[1]]
    )
    pcd_crop = pcd.crop(bbox)


    # 2) 바닥 평면 제거
    try:
        _, inliers = pcd_crop.segment_plane(
            distance_threshold=plane_threshold,
            ransac_n=ransac_n,
            num_iterations=num_iterations
        )
        pcd_nofloor = pcd_crop.select_by_index(inliers, invert=True)
    except RuntimeError:
        pcd_nofloor = pcd_crop
    

    # 3) DBSCAN 클러스터링
    labels = np.array(
        pcd_nofloor.cluster_dbscan(eps=cluster_eps,
                                   min_points=cluster_min_points,
                                   print_progress=False)
    )
    valid = labels >= 0
    if not np.any(valid):
        return pcd_nofloor

    # 4) 상위 num_clusters개 군집 라벨 뽑기
    counts = np.bincount(labels[valid])
    top_labels = np.argsort(counts)[::-1][:num_clusters]

    # 5) 위 라벨들에 속하는 점 인덱스 모두 모아서 반환
    mask = np.isin(labels, top_labels)
    indices = np.where(mask)[0]
    return pcd_nofloor.select_by_index(indices)
    

In [ ]:
def visualize(pcd, title="신발 결과", width=800, height=600):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=title, width=width, height=height)
    opt = vis.get_render_option()
    opt.background_color = (0.1, 0.1, 0.1)
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

In [ ]:
if __name__ == "__main__":
   
    input_dir  = os.getcwd()                     
    output_dir = os.path.join(input_dir, "filtered_results") #결과 저장 폴더 
    os.makedirs(output_dir, exist_ok=True)

    # 2) 폴더 내 모든 .pcd 파일 찾아서 작업 진행 코드 
    for fname in os.listdir(input_dir):
        if not fname.lower().endswith(".pcd"):
            continue

        in_path  = os.path.join(input_dir, fname)
        out_name = os.path.splitext(fname)[0] + "_shoes_result.pcd"
        out_path = os.path.join(output_dir, out_name)

        print(f"\n[Processing] {fname}")
        # 3) 배경 제거 수행 
        pcd      = o3d.io.read_point_cloud(in_path)
        filtered = filter_shoes_both(pcd)

        # 4) 저장
        o3d.io.write_point_cloud(out_path, filtered)
        print(f"[Saved] {out_path}")

        # 5) 시각화
        visualize(filtered, title=f"Filtered: {fname}")

    print("\n모든 파일 처리 완료.")


[Processing] cloud_0_test1.pcd
[Saved] c:\Users\Home Bsw\Desktop\bgrm d435\filtered_results\cloud_0_test1_both_shoes.pcd

[Processing] cloud_180_test1.pcd
[Saved] c:\Users\Home Bsw\Desktop\bgrm d435\filtered_results\cloud_180_test1_both_shoes.pcd

[Processing] cloud_270_test1.pcd
[Saved] c:\Users\Home Bsw\Desktop\bgrm d435\filtered_results\cloud_270_test1_both_shoes.pcd

[Processing] cloud_90_test1.pcd
[Saved] c:\Users\Home Bsw\Desktop\bgrm d435\filtered_results\cloud_90_test1_both_shoes.pcd

모든 파일 처리 완료.
